In [1]:
import os 
from PIL import Image
import numpy as np 
import rasterio as rio
import shutil
from rasterio.enums import Resampling
from rasterio.transform import Affine
import matplotlib.pyplot as plt

In [5]:
make folder of only imagery from these metadatas (8band specfically)
dest_folder = 'Des-imagery'
curr_folder = 'Des-metadata'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

for root, dirs, folders in os.walk(f'/Users/aidanhayes/Desktop/cems-reu-fall/{curr_folder}'):
    for direc in dirs: 
        for file in (os.listdir(f'{curr_folder}/{direc}')):
            # making sure only 8 band imagery is being processed
            if 'SR_8b' in file:
                print(f"Found file: {file}")
                src_file_path = os.path.join(root, direc, file)
                dest_file_path = os.path.join(f'/Users/aidanhayes/Desktop/cems-reu-fall/{dest_folder}', file)
                
                # copying file
                shutil.copy(src_file_path, dest_file_path)
                print(f"Copied {file} to {dest_file_path}")


In [6]:
def resize_pad(image_path, scale_factor, output_path):
    target_height = 256
    target_width = 256

    with rio.open(image_path) as src:
        new_height = int(src.height * scale_factor)
        new_width = int(src.width * scale_factor)

        data = src.read(
            out_shape=(
                src.count,
                new_height,
                new_width
            ),
            resampling=Resampling.bilinear
        )

        transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )

        profile = src.profile
        profile.update({
            'height': new_height,
            'width': new_width,
            'transform': transform
        })

        padded_data = np.zeros((src.count, target_height, target_width), dtype=data.dtype)
        pad_height = (target_height - new_height) // 2
        pad_width = (target_width - new_width) // 2

        padded_data[:, pad_height:pad_height + new_height, pad_width:pad_width + new_width] = data

        profile.update({
            'height': target_height,
            'width': target_width,
            'transform': transform * Affine.translation(-pad_width, -pad_height)
        })

        with rio.open(output_path, 'w', **profile) as dst:
            dst.write(padded_data)

def scale_factor(height, width):
    if height > width:
        num = height / 256
    else:
        num = width / 256
    return 1 / num

In [9]:
# setting variables
folder_path = '/Users/aidanhayes/Desktop/cems-reu-fall'
image_dataset = []

# looping through to get all imagery
for root, dirs, files in os.walk(folder_path):
    for direc in dirs:
        if 'imagery' in direc:
            for filename in os.listdir(direc):
                image_path = os.path.join(direc, filename)
                with rio.open(image_path) as src:
                    bands = [src.read(i) for i in range(1, src.count + 1)]

                # current image dimensions
                image = np.stack(bands, axis=-1)
                # Check image dimensions
                img_height, img_width, num_bands = image.shape
                factor = scale_factor(img_height, img_width)
                
                output_dir = os.path.join(root, direc + '-resized')
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                output_path = os.path.join(output_dir, filename)
                
                # Resizing and rewriting it to site-imagery-resized
                resize_pad(image_path, factor, output_path)  
